In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binku1/hindi_train_sample (1).csv
/kaggle/input/binku-man-1/hindi_train_sample_final.csv


In [2]:
df = pd.read_csv('/kaggle/input/binku1/hindi_train_sample (1).csv')

In [3]:
df.head()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content
0,-1,economy,অর্থনীতি,"০৭ সেপ্টেম্বর ২০১৪, ০০:০১","০৭ সেপ্টেম্বর ২০১৪, ০০:০১",['বাণিজ্য'],0,ইবিএলের হজ ও ওমরাহ কার্ড চালু,http://www.prothom-alo.com/economy/article/312067,ईस्टर्न बैंक लिमिटेड (ईबीएल) ने प्रमुख अंतरराष...
1,খুলনা অফিস,bangladesh,বাংলাদেশ,"২৬ জুন ২০১৬, ০১:৩৯","২৬ জুন ২০১৬, ০২:৩৩","['খুলনা', 'খুলনা বিভাগ', 'অপরাধ', 'বিশাল বাংলা']",৯,শিক্ষককে পেটালেন যুবলীগ নেতা,http://www.prothom-alo.com/bangladesh/article/...,खुलना के दकोप उपजिला जुबो लीग के महासचिव और कै...
2,"প্রতিনিধি, কেরানীগঞ্জ",bangladesh,বাংলাদেশ,"১১ জুলাই ২০১৮, ১১:৩২","১১ জুলাই ২০১৮, ১১:৩৩","['বন্দুকযুদ্ধ', 'অপরাধ', 'কেরানীগঞ্জ', 'ঢাকা ব...",১,কেরানীগঞ্জে ‘বন্দুকযুদ্ধে’ নিহত ২১ মামলার আসামি,http://www.prothom-alo.com/bangladesh/article/...,ढाका के केरानीगंज मॉडल थाना अंतर्गत देवसूर इला...
3,ফরিদপুর অফিস,bangladesh,বাংলাদেশ,"২৩ ফেব্রুয়ারি ২০১৬, ১৮:০৮","২৩ ফেব্রুয়ারি ২০১৬, ১৮:১১",['অপরাধ'],0,দুদিন পর মধুমতীতে ভেসে উঠল লাশ,http://www.prothom-alo.com/bangladesh/article/...,संघर्ष के एक चरण में सिर में चोट लगने के कारण ...
4,খুলনা অফিস,bangladesh,বাংলাদেশ,"২৮ মার্চ ২০১৭, ০০:৩৯","২৮ মার্চ ২০১৭, ০০:৫৭","['খুলনা', 'খুলনা বিভাগ', 'মহানগর']",0,কর্মশালা,http://www.prothom-alo.com/bangladesh/article/...,खुलना विश्वविद्यालय के पांच विभागों (विषयों) क...


In [4]:
print(df['category'].unique())
df.shape

['economy' 'bangladesh' 'international' 'technology' 'entertainment'
 'sports' 'life-style' 'education' 'opinion']


(3523, 10)

In [5]:
df = df[(df['category'] != 'bangladesh')] 
df = df[(df['category'] != 'international')]
df.shape

(1127, 10)

In [6]:
df.category.value_counts()

sports           417
entertainment    303
technology       120
economy          117
opinion           60
life-style        55
education         55
Name: category, dtype: int64

In [7]:
df.content = df.content.str.replace('\d+',' ')

/tmp/ipykernel_20/3576015957.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.content = df.content.str.replace('\d+',' ')


In [8]:
df.category = df.category.replace('sports', 1)
df.category = df.category.replace('technology', 2)
df.category = df.category.replace('entertainment', 3)
df.category = df.category.replace('economy', 4)
df.category = df.category.replace('life-style', 5)
df.category = df.category.replace('opinion', 6)
df.category = df.category.replace('education', 7)

df.category.head()

0     4
9     2
13    3
15    4
19    1
Name: category, dtype: int64

In [9]:
import re
whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
bangla_fullstop = u"\u0964"
punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"

def remove_punct(t1):
    #t1 ="সৌতিরুবাচ।   তেষু তত্রোপবিষ্টেষু ব্রাহ্মণেষু মহাত্মসু। রুরুশ্চুক্রোশ গহনং বনং গৎবাঽতিদুঃখিতঃ॥   শোকেনাভিহতঃ সোঽথ বিলপন্করুণং বহু। অব্রবীদ্বচনং শোচন্প্রিয়াং স্মৃৎবা প্রমদ্বরাম্॥   শেতে সা ভুবি তন্বঙ্গী মম শোকবিবর্ধিনী। `প্রাণানপহরন্তীব পূর্ণচন্দ্রনিভাননা॥   যদি পীনায়তশ্রোণী পদ্মপত্রনিভেক্ষণা। মুমূর্ষুরপি মে প্রাণানাদায়াশু গমিষ্যতি॥   পিতৃমাতৃসখীনাং চ লুপ্তপিণ্ডস্য তস্য মে।' বান্ধবানাং চ সর্বেষাং কিং নু দুঃখমতঃপরম্॥   যদি দত্তং তপস্তপ্তং গুরবো বা ময়া যদি। সম্যগারাধিতাস্তেন সংজীবতু মম প্রিয়া॥   যথা চ জন্মপ্রভৃতি যতাত্মাঽহং ধৃতব্রতঃ। প্রমদ্বরা তথাদ্যৈষা সমুত্তিষ্ঠতু ভামিনী॥   [এবং লালপ্যতস্তস্য ভার্যার্থে দুঃখিতস্য চ। দেবদূতস্তদাঽভ্যেত্য বাক্যমাহ রুরুং বনে॥]   `কৃষ্ণে বিষ্ণৌ হৃষীকেশে লোকেশেঽসুরবিদ্বিষি। যদি মে নিশ্চলা ভক্তির্মম জীবতু সা প্রিয়া॥   বিলপ্যমানে তু রুরৌ সর্বে দেবাঃ কৃপান্বিতাঃ। দূতং প্রস্থাপয়ামাসুঃ সংদিশ্যাস্য হিতং বচঃ॥   স দূতস্ৎবরিতোঽভ্যেত্য দেবানাং প্রিয়কৃচ্ছুচিঃ। উবাচ দেববচনং রুরুমাভাষ্য দুঃখিতম্॥   দেবৈঃ সর্বৈরহং ব্রহ্মন্প্রেষিতোঽস্মি তবান্তিকম্। ৎবদ্ধিতং ৎবদ্ধিতৈরুক্তং শৃণু বাক্যং দ্বিজোত্তম॥'   অভিধৎসে হ যদ্বাচা রুরো দুঃখান্ন তন্মৃষা। ন তু মর্ত্যস্য ধর্মাত্মন্নায়ুরস্তি গতায়ুষঃ॥   গতায়ুরেষা কৃপণা গন্ধর্বাপ্সরসোঃ সুতা। তস্মাচ্ছোকে মনস্তাত মা কৃথাস্ৎবং কথংচন॥   উপায়শ্চাত্র বিহিতঃ পূর্বং দেবৈর্মহাত্মভিঃ। তং যদীচ্ছসি কর্তুং ৎবং প্রাপ্স্যসীহ প্রমদ্বরাম্॥   রুরুরুবাচ।   ক উপায়ঃ কৃতো দেবৈর্বূহি তত্ৎবেন খেচর। করিষ্যেঽহং তথা শ্রুৎবা ত্রাতুমর্হতি মাং ভবান্॥   দেবেদূত উবাচ।   আয়ুষোঽর্ধং প্রয়চ্ছ ৎবং কন্যায়ৈ ভৃগুনন্দন। এবমুত্থাস্যতি রুরো তব ভার্যা প্রয়দ্বরা॥   রুরুরুবাচ।   আয়ুষোঽর্ধং প্রয়চ্ছামি কন্যায়ৈ খেচরোত্তম। শৃঙ্গাররূপাভরণা সমুত্তিষ্ঠতু মে প্রিয়া॥   সৌতিরুবাচ।   ততো গন্ধর্বরাজশ্চ দেবদূতশ্চ সত্তমৌ। ধর্মরাজমুপেত্যেদং বচনং প্রত্যভাষতাম্॥   ধর্মরাজায়ুষোঽর্ধেন রুরোর্ভার্যা প্রমদ্বরা। সমুত্তিষ্ঠতু কল্যাণী মৃতৈবং যদি মন্যসে॥   ধর্মরাজ উবাচ।   প্রমদ্বরা রুরোর্ভার্যা দেবদূত যদীচ্ছসি। উত্তিষ্ঠৎবায়ুষোঽর্ধেন রুরোরেব সমন্বিতা॥   সৌতিরুবাচ।   এবমুক্তে ততঃ কন্যা সোদতিষ্ঠৎপ্রমদ্বরা। রুরোস্তস্যায়ুষোঽর্ধেন সুপ্তেব বরবর্ণিনী॥   এতদ্দৃষ্টং ভবিষ্যে হি রুরোরুত্তমতেজসঃ। আয়ুষোঽতিপ্রবৃদ্ধস্য ভার্যার্থেঽর্ধমলুপ্যত॥   তত ইষ্টেঽহনি তয়োঃ পিতরৌ চক্রতুর্মুদা। বিবাহং তৌ চ রেমাতে পরস্পরহিতৈষিণৌ॥   স লব্ধ্বা দুর্লভাং ভার্যাং পদ্মকিঞ্জল্কসুপ্রভাম্। ব্রতং চক্রে বিনাশায় জিহ্মগানাং ধৃতব্রতঃ॥   স দৃষ্ট্বা জিহ্মগান্সর্বাংস্তীব্রকোপসমন্বিতঃ। অভিহন্তি যথাসত্ৎবং গৃহ্য প্রহরণং সদা॥   স কদাচিদ্বনং বিপ্রো রুরুরভ্যাগমন্মহৎ। শয়ানং তত্র চাপশ্যড্ডুণ্ডুভং বয়সান্বিতম্॥   তত উদ্যম্য দম্ডং স কালদণ্ডোপমং তদা। জিঘাংসুঃ কুপিতো বিপ্রস্তমুবাচাথ ডুণ্ডুভঃ॥   নাপরাধ্যামি তে কিংচিদহমদ্য তপোধন। সংরম্ভাচ্চ কিমর্থং মামভিহংসি রুষান্বিতঃ॥ ॥   ইতি শ্রীমন্মহাভারতে আদিপর্বাণি পৌলোমপর্বণি নবমোঽধ্যায়ঃ॥  ॥"
    t1= whitespace.sub(" ",t1).strip()
    t1 = re.sub(punctSeq, " ", t1)
    t1 = re.sub(bangla_fullstop, " ",t1)
    t1 = re.sub(punc, " ", t1)
    return t1

df['content']=df['content'].apply(lambda x: remove_punct(x))

df.head()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content
0,-1,4,অর্থনীতি,"০৭ সেপ্টেম্বর ২০১৪, ০০:০১","০৭ সেপ্টেম্বর ২০১৪, ০০:০১",['বাণিজ্য'],0,ইবিএলের হজ ও ওমরাহ কার্ড চালু,http://www.prothom-alo.com/economy/article/312067,ईस्टर्न बैंक लिमिटेड ईबीएल ने प्रमुख अंतरराष...
9,অনলাইন ডেস্ক,2,বিজ্ঞান ও প্রযুক্তি,"২০ আগস্ট ২০১৬, ১৩:২৮","২০ আগস্ট ২০১৬, ১৩:৩১","['মোবাইল ফোন', 'খবরাখবর']",১,এ বছরই ফোন নিয়ে ফিরছে নকিয়া,http://www.prothom-alo.com/technology/article/...,नोकिया ब्रांड के स्मार्टफोन और टैबलेट कंप्यूटर...
13,-1,3,বিনোদন,"০৫ নভেম্বর ২০১৩, ০০:০৯","০৫ নভেম্বর ২০১৩, ০০:০৯","['বিনোদন', 'আলাপন']",0,‘চলচ্চিত্রের গানে নিয়মিত কণ্ঠ দিচ্ছি’,http://www.prothom-alo.com/entertainment/artic...,कोनल संगीतकार चैनल आज शाम मन्ना डे स्मृति कार...
15,-1,4,অর্থনীতি,"০৫ সেপ্টেম্বর ২০১৪, ০০:০১","০৫ সেপ্টেম্বর ২০১৪, ০০:০১","['বাণিজ্য সংবাদ', 'বাণিজ্য']",0,চট্টগ্রামে লোটোর বিক্রয়কেন্দ্র চালু,http://www.prothom-alo.com/economy/article/310261,इतालवी ब्रांड लोट्टो का एक प्रमुख आउटलेट या बि...
19,চট্টগ্রাম থেকে প্রতিনিধি,1,খেলা,"২৯ নভেম্বর ২০১৬, ০১:৫১","২৯ নভেম্বর ২০১৬, ০১:৫৪","['খেলা', 'দেশের ফুটবল']",0,একটি ফাউলের সুফল পেল মোহামেডান,http://www.prothom-alo.com/sports/article/1030235,मैदान पर अच्छे दिन अब मोहम्मडन के सामने कालेबड...


In [10]:
'''!pip install bnlp_toolkit

from bnlp import BasicTokenizer
basic_t = BasicTokenizer()
df['content']=df['content'].apply(lambda x: basic_t.tokenize(x))

df.head()'''

"!pip install bnlp_toolkit\n\nfrom bnlp import BasicTokenizer\nbasic_t = BasicTokenizer()\ndf['content']=df['content'].apply(lambda x: basic_t.tokenize(x))\n\ndf.head()"

In [11]:
!pip install nltk

import nltk

# Download the necessary resources for tokenizing Hindi text
nltk.download('punkt')
nltk.download('indian')

# Tokenize Hindi text

df['content'] = df['content'].apply(lambda x:nltk.word_tokenize(x))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package indian to /usr/share/nltk_data...
[nltk_data]   Package indian is already up-to-date!


In [12]:
df.content.head()

0     [ईस्टर्न, बैंक, लिमिटेड, ईबीएल, ने, प्रमुख, अं...
9     [नोकिया, ब्रांड, के, स्मार्टफोन, और, टैबलेट, क...
13    [कोनल, संगीतकार, चैनल, आज, शाम, मन्ना, डे, स्म...
15    [इतालवी, ब्रांड, लोट्टो, का, एक, प्रमुख, आउटले...
19    [मैदान, पर, अच्छे, दिन, अब, मोहम्मडन, के, सामन...
Name: content, dtype: object

In [13]:
!pip install advertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.0/310.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


In [14]:
import advertools as adv
adv.stopwords.keys()

dict_keys(['arabic', 'azerbaijani', 'bengali', 'catalan', 'chinese', 'croatian', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hindi', 'hungarian', 'indonesian', 'irish', 'italian', 'japanese', 'kazakh', 'nepali', 'norwegian', 'persian', 'polish', 'portuguese', 'romanian', 'russian', 'sinhala', 'spanish', 'swedish', 'tagalog', 'tamil', 'tatar', 'telugu', 'thai', 'turkish', 'ukrainian', 'urdu', 'vietnamese'])

In [15]:
stop_words=sorted(adv.stopwords['hindi'])

In [16]:
def remove_stopwords(text):
  words = [w for w in text if w not in set(stop_words)]
  return words

df['content']=df['content'].apply(lambda x:remove_stopwords(x))


df.content.head()

0     [ईस्टर्न, बैंक, लिमिटेड, ईबीएल, प्रमुख, अंतररा...
9     [नोकिया, ब्रांड, स्मार्टफोन, टैबलेट, कंप्यूटर,...
13    [कोनल, संगीतकार, चैनल, आज, शाम, मन्ना, डे, स्म...
15    [इतालवी, ब्रांड, लोट्टो, प्रमुख, आउटलेट, बिक्र...
19    [मैदान, अच्छे, दिन, अब, मोहम्मडन, सामने, कालेब...
Name: content, dtype: object

In [17]:
type(df.content[0])

list

In [18]:
def List(text):
    mod_text = " ".join(text)
    return mod_text

df['content'] = df['content'].apply(lambda x:List(x))

In [19]:
df.content.head()

0     ईस्टर्न बैंक लिमिटेड ईबीएल प्रमुख अंतरराष्ट्री...
9     नोकिया ब्रांड स्मार्टफोन टैबलेट कंप्यूटर मॉडल ...
13    कोनल संगीतकार चैनल आज शाम मन्ना डे स्मृति कार्...
15    इतालवी ब्रांड लोट्टो प्रमुख आउटलेट बिक्री केंद...
19    मैदान अच्छे दिन अब मोहम्मडन सामने कालेबड्रेई आ...
Name: content, dtype: object

In [20]:
corpus = []
corpus = df['content']
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1200,lowercase=False)
X = cv.fit_transform(corpus).toarray()
y = df.category.values


In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [22]:
# training the Naive Bayes Classification model to the training dataset
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

In [23]:
# predicting the values classified by the following model
y_pred = classifier.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.6106194690265486
Confusion Matrix:
 [[69  6  7  1  2  0  2]
 [ 1 12  5  0  1  0  0]
 [12  2 35  2  3  0  1]
 [ 3  5  2 15  1  0  1]
 [ 3  0  4  0  2  0  0]
 [ 6  1  7  0  0  4  1]
 [ 5  1  2  0  0  1  1]]


In [25]:
data = pd.read_csv("/kaggle/input/binku-man-1/hindi_train_sample_final.csv")
data.head()

,Text
0,"संजय ने कहा, ""तब, हे राजन, युद्ध में अजेय महाब..."
1,"""भीष्म ने कहा, 'वहां ऋषिगण, पितरों और देवताओं ..."
2,"Sanjaya said,--""Beholding Dhananjaya then to t..."
3,"धृतराष्ट्र ने कहा, 'हे महान बुद्धिमान, मुझे बत..."
4,"युधिष्ठिर ने कहा, 'हे महाबाहो, मुझे बताओ कि इस..."


In [26]:
data.Text = data.Text.str.replace('\d+',' ')
data['Text']=data['Text'].apply(lambda x: remove_punct(x))
data['Text']=data['Text'].apply(lambda x: nltk.word_tokenize(x))
data['Text']=data['Text'].apply(lambda x:remove_stopwords(x))
data['Text']=data['Text'].apply(lambda x:List(x))

corpus_1=[]
corpus_1=data['Text']

Test_t = cv.fit_transform(corpus_1).toarray()

/tmp/ipykernel_20/3693976742.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.Text = data.Text.str.replace('\d+',' ')


In [27]:
y_pred_hindi = classifier.predict(Test_t)

print(y_pred_hindi)

Y_hindi = pd.DataFrame(y_pred_hindi)

Y_hindi.to_csv('y_pred_hindi_1.csv',index=False)

[1 3 3 1 1 1 3 1 3 1 1 1 1 1 1 3 3 1 1 1 1 1 3 1 1 1 1 1 1 1 3 1 1 1 1 4 1
 1 1 1 1 3 1 1 1 7 1 1 1 1 1 1 5 1 1 1 1 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1
 1 1 3 1 1 1 1 3 1 1 1 1 1 3 3 1 1 1 3 1 1 4 1 1 1 1]
